In [1]:
import vanna as vn
from vanna.remote import VannaDefault

Vanna credentials

In [2]:
api_key = "API-KEY"
vanna_model_name = 'RAG model'

OpenAI credentials

In [3]:
openai_key = 'AI-KEY'
openai_model = 'gpt-4o-mini'
openai_config = {'api_key': openai_key, 'model': openai_model}

Configure Vanna model

In [4]:
from VannaOpenAI import VannaOpenAI

vn = VannaOpenAI(vanna_model=vanna_model_name, vanna_api_key=api_key, config=openai_config)

Connect Vanna to PostgresDB (localhost)

In [5]:
vn.connect_to_postgres(host='host', dbname='dbname', user='user', password='pass', port='5432')

Training with a generic plan

In [6]:
ddl_information = """
create table grocery_inventory_and_sales_dataset
(
    product_id              varchar,
    product_name            varchar,
    catagory                varchar,
    supplier_id             varchar,
    supplier_name           varchar,
    stock_quantity          double precision,
    reorder_level           double precision,
    reorder_quantity        double precision,
    unit_price              varchar,
    date_received           date,
    last_order_date         date,
    expiration_date         date,
    warehouse_location      varchar,
    sales_volume            double precision,
    inventory_turnover_rate double precision,
    status                  varchar
);

alter table grocery_inventory_and_sales_dataset
    owner to postgres;
"""
vn.train(ddl=ddl_information)

Adding ddl: 
create table grocery_inventory_and_sales_dataset
(
    product_id              varchar,
    product_name            varchar,
    catagory                varchar,
    supplier_id             varchar,
    supplier_name           varchar,
    stock_quantity          double precision,
    reorder_level           double precision,
    reorder_quantity        double precision,
    unit_price              varchar,
    date_received           date,
    last_order_date         date,
    expiration_date         date,
    warehouse_location      varchar,
    sales_volume            double precision,
    inventory_turnover_rate double precision,
    status                  varchar
);

alter table grocery_inventory_and_sales_dataset
    owner to postgres;



'3496544-ddl'

In [7]:
train_sample = [
    {
        'question': "Which are our top 5 best-selling products, and what's their current stock status?",
        'query': """
                    SELECT
                        product_name,
                        category,
                        sales_volume,
                        stock_quantity,
                        reorder_level,
                        CASE
                            WHEN stock_quantity <= reorder_level THEN 'Needs Reorder'
                            ELSE 'OK'
                        END as stock_status
                    FROM sales_table.grocery_inventory_and_sales_dataset
                    ORDER BY sales_volume DESC
                    LIMIT 5;
                """
    },
    {
        'question': "What's our inventory value by category?",
        'query': """
                    SELECT
                        category,
                        COUNT(*) as product_count,
                        ROUND(SUM(stock_quantity * CAST(unit_price AS double precision))::numeric, 2) as total_inventory_value,
                        ROUND(AVG(stock_quantity * CAST(unit_price AS double precision))::numeric, 2) as avg_category_value
                    FROM sales_table.grocery_inventory_and_sales_dataset
                    GROUP BY category
                    ORDER BY total_inventory_value DESC;
                """
    },
    {
        'question': "Which products are critically low on stock (below reorder level) and need immediate attention?",
        'query': """
                    SELECT
                        product_name,
                        category,
                        stock_quantity,
                        reorder_level,
                        reorder_quantity,
                        supplier_name,
                        last_order_date
                    FROM sales_table.grocery_inventory_and_sales_dataset
                    WHERE stock_quantity <= reorder_level
                        AND status = 'Active'
                    ORDER BY (stock_quantity - reorder_level);
                """
    },
    {
        'question': "Which products are expiring in the next 30 days?",
        'query': """
                    SELECT
                        product_name,
                        category,
                        stock_quantity,
                        expiration_date,
                        warehouse_location,
                        supplier_name
                    FROM sales_table.grocery_inventory_and_sales_dataset
                    WHERE expiration_date BETWEEN CURRENT_DATE AND (CURRENT_DATE + INTERVAL '30 days')
                        AND stock_quantity > 0
                    ORDER BY expiration_date;
                """
    },
    {
        'question': "What's our inventory turnover performance by category?",
        'query': """
                    SELECT
                        category,
                        ROUND(AVG(inventory_turnover_rate)::numeric, 2) as avg_turnover_rate,
                        ROUND(MIN(inventory_turnover_rate)::numeric, 2) as min_turnover_rate,
                        ROUND(MAX(inventory_turnover_rate)::numeric, 2) as max_turnover_rate,
                        COUNT(*) as number_of_products
                    FROM sales_table.grocery_inventory_and_sales_dataset
                    GROUP BY category
                    ORDER BY avg_turnover_rate DESC;
                """
    },
    {
        'question': "What's the status distribution of our products, and what's their total value?",
        'query': """
                    SELECT
                        status,
                        COUNT(*) as product_count,
                        ROUND(SUM(stock_quantity * CAST(unit_price AS double precision))::numeric, 2) as total_inventory_value,
                        ROUND(AVG(sales_volume)::numeric, 2) as avg_sales_volume
                    FROM sales_table.grocery_inventory_and_sales_dataset
                    GROUP BY status
                    ORDER BY product_count DESC;
                """
    },
    {
        'question': "Which products haven't been ordered in the last 90 days but still have high stock?",
        'query': """
                    SELECT
                        product_name,
                        category,
                        stock_quantity,
                        last_order_date,
                        sales_volume,
                        inventory_turnover_rate
                    FROM sales_table.grocery_inventory_and_sales_dataset
                    WHERE last_order_date < (CURRENT_DATE - INTERVAL '90 days')
                    AND stock_quantity > reorder_level * 2
                    ORDER BY stock_quantity DESC;
                """
    },
    {
        'question': "What's our warehouse distribution of products, and how much value do we hold in each location?",
        'query': """
                    SELECT
                        warehouse_location,
                        COUNT(*) as number_of_products,
                        SUM(stock_quantity) as total_items,
                        ROUND(SUM(stock_quantity * CAST(unit_price AS double precision))::numeric, 2) as total_inventory_value,
                        COUNT(CASE WHEN stock_quantity <= reorder_level THEN 1 END) as products_to_reorder
                    FROM sales_table.grocery_inventory_and_sales_dataset
                    GROUP BY warehouse_location
                    ORDER BY total_inventory_value DESC;
                """
    },
    {
        'question': "Which are our most profitable product categories based on current stock value?",
        'query': """
                    WITH category_metrics AS (
                        SELECT
                            category,
                            COUNT(*) as product_count,
                            SUM(sales_volume) as total_sales,
                            ROUND(SUM(stock_quantity * CAST(unit_price AS double precision))::numeric, 2) as current_inventory_value,
                            ROUND(AVG(inventory_turnover_rate)::numeric, 2) as avg_turnover_rate
                        FROM sales_table.grocery_inventory_and_sales_dataset
                        GROUP BY category
                    )
                    SELECT
                        category,
                        product_count,
                        total_sales,
                        current_inventory_value,
                        avg_turnover_rate,
                        ROUND((total_sales * avg_turnover_rate)::numeric, 2) as estimated_annual_sales_value
                    FROM category_metrics
                    ORDER BY estimated_annual_sales_value DESC;
                """
    }
]

In [8]:
for sample in train_sample:
    question = sample.get('question')
    query = sample.get('query')
    vn.train(question=question, sql=query)